In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# Import AnimalShelter class from py file
from crud import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# Provide username and password
username = "aacuser"
password = "coding321"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={
        'display':'block',
        'margin-left':'auto',
        'margin-right':'auto',
        'width':'35%',
    })],
        href='www.snhu.edu'),
    
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Sam Maclean'))),
    html.Hr(),
    
    
    html.Div(
        dcc.RadioItems(
            id='radioButtons',
            options=[
                {'label':'Water Rescue','value':'waterRescue'},
                {'label':'Mountain or Wilderness Rescue','value':'mwRescue'},
                {'label':'Disaster or Individual Tracking','value':'diTracking'},
                {'label':'Reset','value':'reset'}
            ],
            value='radioButton1'
        )

    ),
    html.Hr(),
    
    
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        # Features for user-friendly sorting and filtering
        sort_action = 'native',
        filter_action = 'native',
        page_action = 'native',
        page_current = 0,
        page_size = 10,
        selected_columns = [],
        selected_rows = [],
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
#@app.callback([Output('datatable-id','data'),
#                Output('datatable-id','columns')],
#                [Input('filter-type', 'value')])
@app.callback(Output('datatable-id','data'),
                [Input('radioButtons', 'value')])
def update_dashboard(filter_type):

        #Display items in list based on radio button selected 
        if filter_type == 'waterRescue':
            df = pd.DataFrame(list(shelter.read({
                "sex_upon_outcome":"Intact Female",
                "breed":{
                    "$in":[
                        "Chesapeake Bay Retriever",
                        "Labrador Retriever Mix",
                        "Newfoundland"
                    ]},
                "$and":[
                {"age_upon_outcome_in_weeks":{
                    "$gte":26
                }},
                {"age_upon_outcome_in_weeks":{
                    "$lte":156
                }}]
                
                })))
            
        elif filter_type == 'mwRescue':
            df = pd.DataFrame(list(shelter.read({
                "sex_upon_outcome":"Intact Male",
                "breed":{
                    "$in":[
                        "German Shepherd",
                        "Alaskan Malamute",
                        "Old English Sheepdog",
                        "Siberian Husky",
                        "Rottweiler"
                    ]},
                "$and":[
                {"age_upon_outcome_in_weeks":{
                    "$gte":26
                }},
                {"age_upon_outcome_in_weeks":{
                    "$lte":156
                }}]
                
                })))
            
        elif filter_type == 'diTracking':
            df = pd.DataFrame(list(shelter.read({
                "sex_upon_outcome":"Intact Male",
                "breed":{
                    "$in":[
                        "Doberman Pinscher",
                        "German Shepherd",
                        "Golden Retriever",
                        "Bloodhound",
                        "Rottweiler"
                    ]},
                "$and":[
                {"age_upon_outcome_in_weeks":{
                    "$gte":20
                }},
                {"age_upon_outcome_in_weeks":{
                    "$lte":300
                }}]
                
                })))
                              
 
                              
        elif filter_type == 'reset':
            df = pd.DataFrame(list(shelter.read({})))   
                              
        #columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
                              
        #return(data,columns)
        return(data)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    dff = pd.DataFrame.from_dict(viewData)
    #dfPie = dff.breed.value_counts
    
    #Get names and breeds from database
    names = dff['breed'].value_counts().keys().to_list()
    breeds = dff['breed'].value_counts().keys().to_list()
    
    
    # add code for chart of your choice (e.g. pie chart) #
    return [
        dcc.Graph(
            #Return the data as a pie chart
            figure = px.pie(
                dff,
                #names = 'animal_name'
                names = 'breed'
            )
        )
    ]

#@app.callback(
#    Output('map-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "selected_rows")]
)

#Geolocation chart function
def update_map(viewData, selected_rows):
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[(dff.iloc[selected_rows[0],13]),(dff.iloc[selected_rows[0],14])]
                      , children=[
                dl.Tooltip(dff.iloc[selected_rows[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[selected_rows[0],9])
                ])
            ])
        ])
    ]





app